In [1]:
%matplotlib inline

import os, sys, tqdm
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread, imsave
import pandas as pd
import pyclesperanto_prototype as cle
import napari

sys.path.append('utils')
import register_icp as icp2
import icp_known_correspondence as icp1

from importlib import reload
reload(icp1)
reload(icp2)

<module 'register_icp' from 'utils\\register_icp.py'>

In [ ]:
folders = [
#     os.path.join('..','HCR_time_course','hpf08','Panel1','2021-09-20_084304'), # reference for hpf08
#     os.path.join('..','HCR_time_course','hpf08','Panel1','2021-09-20_084954'), # 
#     os.path.join('..','HCR_time_course','hpf08','Panel1','2021-09-20_085804'), # 
    

#     os.path.join('..','HCR_time_course','hpf10','Panel1','2021-09-20_104647'), # reference for hpf10
# #     os.path.join('..','HCR_time_course','hpf10','Panel1','2021-09-20_104949'), # 
# #     os.path.join('..','HCR_time_course','hpf10','Panel1','2021-09-20_105356'), # 
    
# #     os.path.join('..','HCR_time_course','hpf10','Panel2','2021-10-26_072116'),
# #     os.path.join('..','HCR_time_course','hpf10','Panel2','2021-10-26_072414'),
    
# #     os.path.join('..','HCR_time_course','hpf10','Panel3','2021-10-26_101501'),
# #     os.path.join('..','HCR_time_course','hpf10','Panel3','2021-10-26_101721'),
#     os.path.join('..','HCR_time_course','hpf10','Panel3','2021-10-26_102004'),

# #     os.path.join('..','HCR_time_course','hpf10','Panel4','2021-10-26_104907'),
# #     os.path.join('..','HCR_time_course','hpf10','Panel4','2021-10-26_105128'),


     os.path.join('..','HCR_time_course','hpf12','Panel1','2021-09-20_110602'), # reference for hpf12
# #     os.path.join('..','HCR_time_course','hpf12','Panel1','2021-09-20_105648'), # 

# #     os.path.join('..','HCR_time_course','hpf12','Panel2','2021-10-26_104331'), 

# #     os.path.join('..','HCR_time_course','hpf12','Panel3','2021-10-26_110047'),  
# #     os.path.join('..','HCR_time_course','hpf12','Panel3','2021-10-26_110714'),  

     os.path.join('..','HCR_time_course','hpf12','Panel4','2021-10-26_111407'),  
# #     os.path.join('..','HCR_time_course','hpf12','Panel4','2021-10-26_111929'),  
]

markers_dict = {'Panel1':'nkx2.4b','Panel2':'foxa2','Panel3':'ta','Panel4':'prdm1'}
markers = [markers_dict[folder.split(os.sep)[-2]] for folder in folders]
print(markers)

# 0. Import data and visualize

In [ ]:
fluos_dfs_original = [
    pd.read_csv(os.path.join(i,'cells_segmented','cell_fluo.csv')) for i in folders
    
]

fluos_dfs = [i[i.density>50] for i in fluos_dfs_original]
# fluos_dfs = [i.nlargest(15000, 'density') for i in fluos_dfs]
# fluos_dfs = [i.sample(10000) for i in fluos_dfs_original]


In [ ]:
# %matplotlib notebook

fig = plt.figure(figsize=(8,8))
# fig.suptitle(gene_markers[i], fontsize=16)
ax = fig.add_subplot(projection='3d')



# ax.imshow(img_gauss[z])
ax.scatter(fluos_dfs[0].x_unit, fluos_dfs[0].y_unit, fluos_dfs[0].z_unit, alpha=0.1, c=fluos_dfs[0][markers[0]], cmap='hot')
# ax.scatter(fluos_dfs[1].z_unit,fluos_dfs[1].y_unit,fluos_dfs[1].x_unit,alpha=0.1)
ax.scatter(fluos_dfs[1].x_unit,fluos_dfs[1].y_unit,fluos_dfs[1].z_unit,alpha=0.1, c=fluos_dfs[1][markers[1]], cmap='viridis')

ax.set_box_aspect([1,1,1])
#     set_axes_equal(ax))

# 1. Perform ICP registration

## 1.1 Manually select matching points in the two datasets to make coarse registration

### 1.1.1 Interactively select the points for coarse registration

In [ ]:
point_list = [[],[]]

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

dataset_index=0

# taken from: https://stackoverflow.com/questions/58218482/annotate-3d-scatter-plot-on-pick-event
def getxyz(event):
#     print(event.mouseevent)

    if event.mouseevent.button==3:
        if len(point_list[dataset_index])>0:
            point_list[dataset_index].pop()
            print("Removed last point: ListLen:%d"%len(point_list[dataset_index]))
        return
    if event.mouseevent.button!=1:
        print('Not left click!')
        return
#     if len(event.ind)>1:
#         print('Multiple points selected!')
#         return
    
    xx=event.mouseevent.x
    yy=event.mouseevent.y
    
    #magic from https://stackoverflow.com/questions/10374930/matplotlib-annotating-a-3d-scatter-plot
    x2, y2, z2=proj3d.proj_transform(x[0], y[0], z[0], plt.gca().get_proj())
    x3, y3 = ax.transData.transform((x2, y2))
    d=np.sqrt ((x3 - xx)**2 + (y3 - yy)**2)
    
    #find the closest by searching for min distance.
    #All glory to https://stackoverflow.com/questions/10374930/matplotlib-annotating-a-3d-scatter-plot
    imin=0
    dmin=10000000
    for i in range(len(x)):
        #magic from https://stackoverflow.com/questions/10374930/matplotlib-annotating-a-3d-scatter-plot
        x2, y2, z2=proj3d.proj_transform(x[i], y[i], z[i], plt.gca().get_proj())
        x3, y3 = ax.transData.transform((x2, y2))
        #the distance magic from https://stackoverflow.com/questions/10374930/matplotlib-annotating-a-3d-scatter-plot
        d=np.sqrt ((x3 - xx)**2 + (y3 - yy)**2)
        #We find the distance and also the index for the closest datapoint
        if d< dmin:
            dmin=d
            imin=i

    point_list[dataset_index].append([x[imin], y[imin], z[imin]])
    print("ListLen:%d"%len(point_list[dataset_index]),"X:%.2f"%x[imin], " Y:%.2f"%y[imin], " Z:%.2f"%z[imin], " PointIdx:", imin)

    return

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(projection='3d')

a = fluos_dfs[dataset_index].sample(5000)
a = a[['x_unit','y_unit','z_unit',markers[0]]].to_numpy()
x = a[:,0]
y = a[:,1]
z = a[:,2]
scatter = ax.scatter(x,y,z,alpha=1,c=a[:,3],cmap='hot',picker=True)

# i=0
# for p in a:
#     text = str(i) + ':' + str(p[0])[:5] + ', ' + str(p[1])[:5] + ', ' + str(p[2])[:5]
#     ax.text(p[0],p[1],p[2], text, fontsize=5)
#     i+=1

fig.canvas.mpl_connect('pick_event', getxyz)

ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)

ax.set_xlabel('X ax')
ax.set_ylabel('Y ax')
ax.set_zlabel('Z ax')
ax.set_box_aspect([1,1,1])


In [ ]:
%matplotlib qt
from mpl_toolkits.mplot3d import proj3d

dataset_index=1

# taken from: https://stackoverflow.com/questions/58218482/annotate-3d-scatter-plot-on-pick-event
def getxyz(event):
#     print(event.mouseevent)

    if event.mouseevent.button==3:
        if len(point_list[dataset_index])>0:
            point_list[dataset_index].pop()
            print("Removed last point: ListLen:%d"%len(point_list[dataset_index]))
        return
    if event.mouseevent.button!=1:
        print('Not left click!')
        return
#     if len(event.ind)>1:
#         print('Multiple points selected!')
#         return
    
    xx=event.mouseevent.x
    yy=event.mouseevent.y
    
    #magic from https://stackoverflow.com/questions/10374930/matplotlib-annotating-a-3d-scatter-plot
    x2, y2, z2=proj3d.proj_transform(x[0], y[0], z[0], plt.gca().get_proj())
    x3, y3 = ax.transData.transform((x2, y2))
    d=np.sqrt ((x3 - xx)**2 + (y3 - yy)**2)
    
    #find the closest by searching for min distance.
    #All glory to https://stackoverflow.com/questions/10374930/matplotlib-annotating-a-3d-scatter-plot
    imin=0
    dmin=10000000
    for i in range(len(x)):
        #magic from https://stackoverflow.com/questions/10374930/matplotlib-annotating-a-3d-scatter-plot
        x2, y2, z2=proj3d.proj_transform(x[i], y[i], z[i], plt.gca().get_proj())
        x3, y3 = ax.transData.transform((x2, y2))
        #the distance magic from https://stackoverflow.com/questions/10374930/matplotlib-annotating-a-3d-scatter-plot
        d=np.sqrt ((x3 - xx)**2 + (y3 - yy)**2)
        #We find the distance and also the index for the closest datapoint
        if d< dmin:
            dmin=d
            imin=i

    point_list[dataset_index].append([x[imin], y[imin], z[imin]])
    print("ListLen:%d"%len(point_list[dataset_index]),"X:%.2f"%x[imin], " Y:%.2f"%y[imin], " Z:%.2f"%z[imin], " PointIdx:", imin)

    return

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(projection='3d')

a = fluos_dfs[dataset_index].sample(5000)
a = a[['x_unit','y_unit','z_unit',markers[1]]].to_numpy()
x = a[:,0]
y = a[:,1]
z = a[:,2]
scatter = ax.scatter(x,y,z,alpha=1,c=a[:,3],cmap='hot',picker=True)

# i=0
# for p in a:
#     text = str(i) + ':' + str(p[0])[:5] + ', ' + str(p[1])[:5] + ', ' + str(p[2])[:5]
#     ax.text(p[0],p[1],p[2], text, fontsize=5)
#     i+=1

fig.canvas.mpl_connect('pick_event', getxyz)

ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)

ax.set_xlabel('X ax')
ax.set_ylabel('Y ax')
ax.set_zlabel('Z ax')
ax.set_box_aspect([1,1,1])


In [ ]:
point_list = np.array(point_list)
print(point_list)
print(point_list.shape)

### 1.1.2 Make a first coarse registration

In [ ]:
R_coarse, T_coarse = icp1.icp_known_correspondence(point_list[1],point_list[0])
result = (T_coarse + np.matmul(R_coarse,point_list[1].transpose())).transpose()

if np.allclose(result.transpose(),point_list[0].transpose(), rtol=50):
    print("transformation is correct!")
else:
    print("transformation is wrong...")
    
%matplotlib notebook
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection='3d')
s1=ax.scatter(point_list[0,:,0],point_list[0,:,1],point_list[0,:,2], alpha=1, c='blue')
s2=ax.scatter(point_list[1,:,0],point_list[1,:,1],point_list[1,:,2], alpha=1, c='red')
s3=ax.scatter(result[:,0],result[:,1],result[:,2], alpha=1, c='green')
for i in range(len(result)):
    ax.plot([point_list[1,i,0],result[i,0]],
           [point_list[1,i,1],result[i,1]],
           [point_list[1,i,2],result[i,2]],c='k')
ax.legend([s1,s2,s3],['target','source','moved'])
ax.set_box_aspect([1,1,1])

In [ ]:
### Perform ICP

b = fluos_dfs_original[1][['x_unit','y_unit','z_unit']].to_numpy()
b = (T_coarse + np.matmul(R_coarse,b.transpose())).transpose()

fluos_dfs_original[1]['x_unit'] = b[:,0]
fluos_dfs_original[1]['y_unit'] = b[:,1]
fluos_dfs_original[1]['z_unit'] = b[:,2]

%matplotlib notebook

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection='3d')

ax.scatter(fluos_dfs_original[0].x_unit, 
           fluos_dfs_original[0].y_unit, 
           fluos_dfs_original[0].z_unit, 
           alpha=0.01, c=fluos_dfs_original[0][markers[0]], cmap='hot')
ax.scatter(fluos_dfs_original[1].x_unit, 
           fluos_dfs_original[1].y_unit, 
           fluos_dfs_original[1].z_unit, 
           alpha=0.1, c=fluos_dfs_original[1][markers[1]], cmap='viridis')

ax.set_box_aspect([1,1,1])


## 1.2 Perform finer registration using 1000 points

In [ ]:
### compute transformation with ICP - select 1000 random points from both datasets
reload(icp2)

new_dfs = [i.sample(1000) for i in fluos_dfs_original]

a = new_dfs[0][['x_unit','y_unit','z_unit']].to_numpy()
b = new_dfs[1][['x_unit','y_unit','z_unit']].to_numpy()

T,_,_,_ = icp2.icp(b, a, max_iterations=1000, tolerance=1e-45)
# T = funcs.simpleicp(a, b, correspondences=10000, neighbors=10, min_planarity=0.3, min_change=.1,
#               max_iterations=1000)

### Perform ICP

A = fluos_dfs_original[1][['x_unit','y_unit','z_unit']].to_numpy()
A = icp2.cloud_transform(A, T)

fluos_dfs_original[1]['x_unit'] = A[:,0]
fluos_dfs_original[1]['y_unit'] = A[:,1]
fluos_dfs_original[1]['z_unit'] = A[:,2]

In [ ]:
%matplotlib notebook

fig = plt.figure(figsize=(18,18))
# fig.suptitle(gene_markers[i], fontsize=16)
ax = fig.add_subplot(111,projection='3d')

# ax.imshow(img_gauss[z])
ax.scatter(fluos_dfs_original[0].x_unit,
           fluos_dfs_original[0].y_unit,
           fluos_dfs_original[0].z_unit,
           alpha=0.1, 
           c=fluos_dfs_original[0][markers[0]], cmap='hot')
# ax.scatter(fluos_dfs[1].z_unit,fluos_dfs[1].y_unit,fluos_dfs[1].x_unit,alpha=0.1)
ax.scatter(fluos_dfs_original[1].x_unit,
           fluos_dfs_original[1].y_unit,
           fluos_dfs_original[1].z_unit,
           alpha=0.1, 
           c=fluos_dfs_original[1][markers[1]], cmap='viridis')

ax.set_box_aspect([1,1,1])
ax.set_box_aspect([1,1,1])
#     set_axes_equal(ax))


## 1.3 Save data

In [ ]:
fluos_dfs_original[0].to_csv(os.path.join(folders[0], 'cells_segmented','cell_fluo_reg.csv'), index=False)
fluos_dfs_original[1].to_csv(os.path.join(folders[1], 'cells_segmented','cell_fluo_reg.csv'), index=False)

## 1.4 Visualize all data from a single developmental point

In [ ]:
fluos_dfs = [
    pd.read_csv(os.path.join(i,'cells_segmented','cell_fluo_reg.csv')) for i in folders    
]

# fluos_dfs = [i[i.density>50] for i in fluos_dfs]
# fluos_dfs = [i.sample(5000) for i in fluos_dfs]
n=len(fluos_dfs)

markers = ['density','hand2','emx3','sox10','sp5l','nkx2.4b']

In [ ]:
%matplotlib notebook

marker = 'density'
fig = plt.figure(figsize=(8,3))
fig.suptitle(marker, fontsize=16)
ax = [fig.add_subplot(1,n,i+1,projection='3d') for i in range(n)]

cmap=['hot','viridis']

# ax.imshow(img_gauss[z])
for i in range(n):
    ax[i].scatter( fluos_dfs[i].x_unit, fluos_dfs[i].y_unit, fluos_dfs[i].z_unit, 
                  alpha=0.8, 
                  c=fluos_dfs[i][marker], 
                  cmap=cmap[i] )
    ax[i].set_xlim(-1,1)
    ax[i].set_ylim(-1,1)
    ax[i].set_zlim(-1,1)
    ax[i].set_box_aspect([1,1,1])
    ax[i].set_title('Fish%d'%i)


In [ ]:
# %matplotlib notebook

fig = plt.figure(figsize=(8,8))
# fig.suptitle(gene_markers[i], fontsize=16)
ax = fig.add_subplot(projection='3d')

# ax.imshow(img_gauss[z])
ax.scatter(fluos_dfs[0].x_unit, fluos_dfs[0].y_unit, fluos_dfs[0].z_unit, alpha=0.7, c=fluos_dfs[0]['density'], cmap='hot')
# ax.scatter(fluos_dfs[1].z_unit,fluos_dfs[1].y_unit,fluos_dfs[1].x_unit,alpha=0.1)
ax.scatter(fluos_dfs[1].x_unit,fluos_dfs[1].y_unit,fluos_dfs[1].z_unit,alpha=0.7, c=fluos_dfs[1]['density'], cmap='viridis')

ax.set_box_aspect([1,1,1])
#     set_axes_equal(ax))

# 2. Perform inhomogenous warping with Vedo

In [2]:
import vedo

In [3]:
folders = [
#     os.path.join('..','HCR_time_course','hpf08','Panel1','2021-09-20_084304'), # reference for hpf08
#     os.path.join('..','HCR_time_course','hpf08','Panel1','2021-09-20_084954'), # 
#     os.path.join('..','HCR_time_course','hpf08','Panel1','2021-09-20_085804'), # 
    

#     os.path.join('..','HCR_time_course','hpf10','Panel1','2021-09-20_104647'), # reference for hpf10
#     os.path.join('..','HCR_time_course','hpf10','Panel1','2021-09-20_104949'), # 
#     os.path.join('..','HCR_time_course','hpf10','Panel1','2021-09-20_105356'), # 
    
#     os.path.join('..','HCR_time_course','hpf10','Panel2','2021-10-26_072116'),
#     os.path.join('..','HCR_time_course','hpf10','Panel2','2021-10-26_072414'),
    
#     os.path.join('..','HCR_time_course','hpf10','Panel3','2021-10-26_101501'),
#     os.path.join('..','HCR_time_course','hpf10','Panel3','2021-10-26_101721'),
#     os.path.join('..','HCR_time_course','hpf10','Panel3','2021-10-26_102004'),

#     os.path.join('..','HCR_time_course','hpf10','Panel4','2021-10-26_104907'),
#     os.path.join('..','HCR_time_course','hpf10','Panel4','2021-10-26_105128'),


    os.path.join('..','HCR_time_course','hpf12','Panel1','2021-09-20_110602'), # reference for hpf12
#     os.path.join('..','HCR_time_course','hpf12','Panel1','2021-09-20_105648'), # 

#     os.path.join('..','HCR_time_course','hpf12','Panel2','2021-10-26_104331'), 

#     os.path.join('..','HCR_time_course','hpf12','Panel3','2021-10-26_110047'),  
#     os.path.join('..','HCR_time_course','hpf12','Panel3','2021-10-26_110714'),  

#     os.path.join('..','HCR_time_course','hpf12','Panel4','2021-10-26_111407'),  
    os.path.join('..','HCR_time_course','hpf12','Panel4','2021-10-26_111929'),  
]

markers_dict = {'Panel1':'nkx2.4b','Panel2':'foxa2','Panel3':'ta','Panel4':'prdm1'}
markers = [markers_dict[folder.split(os.sep)[-2]] for folder in folders]
print(markers)

['nkx2.4b', 'prdm1']


In [4]:
fluos_dfs_original = [
    pd.read_csv(os.path.join(i,'cells_segmented','cell_fluo_reg.csv')) for i in folders    
]

fluos_dfs = [i[i.density>50] for i in fluos_dfs_original]
# fluos_dfs = [i.sample(5000) for i in fluos_dfs]
n=len(fluos_dfs)


In [5]:
# define anchor points as average on a grid

point1 = fluos_dfs[0][['x_unit','y_unit','z_unit']].to_numpy()
point1_color = fluos_dfs[0][[markers[0]]].to_numpy().flatten()
point2 = [fluos_dfs[1][['x_unit','y_unit','z_unit']].to_numpy()]
point2_color = fluos_dfs[1][[markers[1]]].to_numpy().flatten()

point1_o = fluos_dfs_original[0][['x_unit','y_unit','z_unit']].to_numpy()
point1_o_color = fluos_dfs_original[0][[markers[0]]].to_numpy()
point2_o = [fluos_dfs_original[1][['x_unit','y_unit','z_unit']].to_numpy()]
point2_o_color = fluos_dfs_original[1][[markers[1]]].to_numpy()

grid_size = .5
pbar = tqdm.tqdm(range(20))
for n in pbar:
    pbar.set_description("grid_size: %.3f"%grid_size)
#     print(grid_size)
    bins = np.arange(-1,1.1,grid_size)
    grid_size = grid_size*0.95
    n_min = 20

    source = []
    target = []

    a = []
    for i in range(len(bins)-1):
        for j in range(len(bins)-1):
            for k in range(len(bins)-1):
                p_s = point2_o[-1]
                s = p_s[(p_s[:,0]>bins[i])&(p_s[:,0]<=bins[i+1])&
                        (p_s[:,1]>bins[j])&(p_s[:,1]<=bins[j+1])&
                        (p_s[:,2]>bins[k])&(p_s[:,2]<=bins[k+1])]

                p_t = point1_o
                t = p_t[(p_t[:,0]>bins[i])&(p_t[:,0]<=bins[i+1])&
                        (p_t[:,1]>bins[j])&(p_t[:,1]<=bins[j+1])&
                        (p_t[:,2]>bins[k])&(p_t[:,2]<=bins[k+1])]

                if (s.shape[0]>n_min)&(t.shape[0]>n_min):
                    source.append(np.mean(s,0))
                    target.append(np.mean(t,0))
    source = np.array(source)
    target = np.array(target)

    p2 = vedo.Points(point2_o[-1]).c('green4')
    newpt2 = p2.clone().warp(source, target, mode='3d', sigma=1)
    point2_o.append(newpt2.points())

    p2 = vedo.Points(point2[-1]).c('green4')
    newpt2 = p2.clone().warp(source, target, mode='3d', sigma=1)
    point2.append(newpt2.points())

point2 = np.array(point2)
point2_o = np.array(point2_o)

grid_size: 0.189: 100%|████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.83it/s]


In [ ]:
# vedo.show([p1,p2], N=2, axes=1)

# show in 2 separate renderers:
# arrows = vedo.Arrows(source, target, s=2).c('k')

# vedo.show([[p1,p2], [p1,arrows], [p1, newpt1]], N=3, axes=1)

In [ ]:
# %matplotlib notebook

# fig = plt.figure(figsize=(16,8))
# # fig.suptitle(gene_markers[i], fontsize=16)
# ax = fig.add_subplot(111,projection='3d')

# p1 = point1
# p1_c = point1_color
# p2 = point2
# p2_c = point2_color

# s1 = ax.scatter(p1[::4,0],
#            p1[::4,1],
#            p1[::4,2],
#            alpha=0.5, 
# #            c=p1_c[::4], 
# #            cmap='hot'
#                   )

# s2 = ax.scatter(p2[0][::4,0],
#            p2[0][::4,1],
#            p2[0][::4,2],
#            alpha=0.1, 
# #            c=p2_c[::4], 
# #            cmap='viridis'
#                   )

# s3 = ax.scatter(p2[-1][::4,0],
#            p2[-1][::4,1],
#            p2[-1][::4,2],
#            alpha=0.5, 
# #            c=p2_c[::4], 
# #            cmap='viridis'
#                   )

# ax.legend([s1, s2, s3], ['target', 'source', 'moved'])

# ax.set_xlim(-1,1)
# ax.set_ylim(-1,1)
# ax.set_zlim(-1,1)

# ax.set_box_aspect([1,1,1])
# ax.view_init(elev=0., azim=-40)


In [ ]:
# %matplotlib notebook

# fig = plt.figure(figsize=(16,8))
# # fig.suptitle(gene_markers[i], fontsize=16)
# ax = fig.add_subplot(111,projection='3d')

# s1 = ax.scatter(point1_o[:,0],
#            point1_o[:,1],
#            point1_o[:,2],
#            alpha=0.5, 
# #            c=fluos_dfs[0]['density'], 
# #            cmap='hot'
#                   )

# # s2 = ax.scatter(point2_o[0][:,0],
# #            point2_o[0][:,1],
# #            point2_o[0][:,2],
# #            alpha=0.5, 
# # #            c=fluos_dfs[0]['density'], 
# # #            cmap='hot'
# #                   )

# s3 = ax.scatter(point2_o[-1][:,0],
#            point2_o[-1][:,1],
#            point2_o[-1][:,2],
#            alpha=0.5, 
# #            c=fluos_dfs[0]['density'], 
# #            cmap='hot'
#                   )

# ax.legend([s1, s2, s3], ['target', 'source', 'moved'])

# ax.set_xlim(-1,1)
# ax.set_ylim(-1,1)
# ax.set_zlim(-1,1)

# ax.set_box_aspect([1,1,1])
# ax.view_init(elev=10., azim=-40)


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import pandas as pd

p2 = [point2_o[0]]
for p in point2_o:
    p2.append(p)
for i in range(10):
    p2.append(point2_o[-1])
        
down = 5

p2_color = point2_o_color[::down]

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1,1,1,projection='3d')

s1 = ax.scatter(point1_o[::down,0],
           point1_o[::down,1],
           point1_o[::down,2],
           alpha=0.5, 
#            c=fluos_dfs[0]['density'], 
#            cmap='hot'
                  )

# s3 = ax.scatter(p2[0][::4,0],
#            p2[0][::4,1],
#            p2[0][::4,2],
#            alpha=.2, 
# #            c=fluos_dfs[0]['density'], 
# #            cmap='hot'
#                   )

s2 = ax.scatter(p2[0][::down,0],
           p2[0][::down,1],
           p2[0][::down,2],
           alpha=.4, 
           c=p2_color, 
           cmap='hot'
                  )


ax.legend([s1, s3, s2], ['target', 'source', 'moving'])

ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
ax.set_zlim(-1,1)

ax.set_box_aspect([1,1,1])
ax.view_init(elev=10., azim=-10)

def update(frame):
    s2._offsets3d = (p2[frame][::down,0], p2[frame][::down,1], p2[frame][::down,2])
#     s2.set_array(p2_color)
    
ani = matplotlib.animation.FuncAnimation(fig, update, len(p2), 
                               interval=100, blit=False)

# from matplotlib import animation
# writervideo = animation.FFMpegWriter(fps=6) 
# ani.save(os.path.join('vedo_registration.avi'), writer=writervideo)


In [ ]:
df_save = fluos_dfs_original[0].copy()
df_save.to_csv(os.path.join(folders[0], 'cells_segmented','cell_fluo_WarpReg.csv'), index=False)
df_save = fluos_dfs_original[1].copy()
df_save.x_unit = point2_o[-1,:,0]
df_save.y_unit = point2_o[-1,:,1]
df_save.z_unit = point2_o[-1,:,2]
df_save.to_csv(os.path.join(folders[1], 'cells_segmented','cell_fluo_WarpReg.csv'), index=False)